In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import randint



In [57]:
# Loda data
target = pd.read_csv('../data/processed/properties.csv')
features = pd.read_csv('../data/processed/features.csv')

# Merge data
df = pd.merge(target, features, on='Period')

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Period         171 non-null    object 
 1   num_nodes      171 non-null    int64  
 2   num_edges      171 non-null    int64  
 3   density        171 non-null    float64
 4   clustering     171 non-null    float64
 5   assortativity  171 non-null    float64
 6   shortest_path  170 non-null    float64
 7   TPU            171 non-null    float64
 8   GPR            171 non-null    float64
 9   GEPU           171 non-null    float64
dtypes: float64(7), int64(2), object(1)
memory usage: 13.5+ KB


In [50]:
df

,Period,num_nodes,num_edges,density,clustering,assortativity,shortest_path,TPU,GPR,DMGSRMRCHSACD,IPTOTSAKD,IMPCOV,NEER,REER,RETSALESSA,TOTRESV,UNEMPSA_,etf_price
0,2010-01-01,168,1828,0.130311,0.712998,-0.444260,2.078985,24.490808,91.581024,1.802235e+06,1.279704e+12,4.880311,99.710300,99.882075,100.970001,8.795469e+06,8.266556,12.225553
1,2010-02-01,172,1838,0.124983,0.760005,-0.468762,2.061200,18.847337,80.725357,1.853262e+06,1.283712e+12,4.751241,100.058497,100.144656,101.121526,8.805293e+06,8.206897,13.201204
2,2010-03-01,171,1923,0.132301,0.776834,-0.466797,2.032542,28.556673,74.116943,1.819304e+06,1.300778e+12,4.882158,99.910024,99.868581,102.100177,8.882128e+06,8.188579,14.057395
3,2010-04-01,170,1850,0.128785,0.713454,-0.467922,2.069474,25.487728,88.761581,1.811889e+06,1.309634e+12,4.978065,99.826979,99.736114,95.659314,9.019701e+06,8.210219,14.341842
4,2010-05-01,167,1876,0.135344,0.773232,-0.462863,2.023086,21.410693,88.958710,1.774453e+06,1.319176e+12,5.057788,100.486460,100.390645,96.108964,8.974806e+06,8.116854,13.403166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,2023-11-01,178,2090,0.132673,0.747174,-0.484728,2.023805,40.948276,156.487473,3.106750e+06,1.881066e+12,4.309817,103.409761,104.427667,100.680543,1.339346e+07,5.661176,170.488037
167,2023-12-01,174,2001,0.132948,0.755731,-0.486585,2.008504,65.307307,142.074020,3.007755e+06,1.887681e+12,4.549854,103.334684,103.834920,100.591706,1.368484e+07,5.670675,191.216141
168,2024-01-01,172,1836,0.124847,0.709186,-0.474903,2.083095,69.553994,159.757202,3.105054e+06,1.879627e+12,4.381012,103.657168,103.853338,101.811391,1.360344e+07,5.686735,194.902161
169,2024-02-01,175,1883,0.123678,0.725464,-0.488365,2.085714,62.485712,146.540665,3.173076e+06,1.890715e+12,4.278387,103.982254,104.100837,102.086548,1.357565e+07,5.709206,216.865021


In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# Drop rows with missing values in the relevant columns
#df = df[['GEPU', 'shortest_path']].dropna()

# Regression Analysis
# Example analysis for 'num_nodes' vs 'density'
# X = df[['TPU','GPR',DMGSRMRCHSACD, IPTOTSAKD, IMPCOV, NEER, REER, RETSALESSA, TOTRESV, UNEMPSA_
X = df[['GEPU']]
Y = df['num_nodes']
# Y = df['num_nodes', 'num_edges''density','clustering','assortativity','shortest_path']

X_const = sm.add_constant(X)
model = sm.OLS(Y, X_const)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              num_nodes   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                   0.07070
Date:                Tue, 30 Jul 2024   Prob (F-statistic):              0.791
Time:                        13:26:28   Log-Likelihood:                -462.39
No. Observations:                 171   AIC:                             928.8
Df Residuals:                     169   BIC:                             935.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        174.7161      0.788    221.621      0.0